In [30]:
# Import required packages 

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr
from scipy.spatial import distance
import seaborn as sns

pd.merge?

In [48]:
# Import MovieLens dataset into pandas DataFrame
cols1 = ['userId', 'movieId', 'rating']
cols2 = [ 'movieId', 'title']

data = pd.read_csv(r'C:/Users/Korisnik/ml-latest-small/ratings.csv', usecols = cols1)

movietitles = pd.read_csv(r'C:/Users/Korisnik/ml-latest-small/movies.csv', low_memory = False, usecols = cols2)

data = pd.merge(data, movietitles, on = 'movieId')
data

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)
...,...,...,...,...
100831,610,160341,2.5,Bloodmoon (1997)
100832,610,160527,4.5,Sympathy for the Underdog (1971)
100833,610,160836,3.0,Hazard (2005)
100834,610,163937,3.5,Blair Witch (2016)


In [49]:
# Split data into Training Data and Test Data

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [50]:
print('Training Data has {} rows, and Test Data has {} rows'.format(train_data.shape[0], test_data.shape[0]))

Training Data has 80668 rows, and Test Data has 20168 rows


In [51]:
# Make ratings tables from Training Data 

ratings = train_data.pivot_table(index = 'userId', columns = 'movieId', values = 'rating')
ratings

movieId,1,2,3,4,5,6,7,8,9,10,...,190221,191005,193565,193567,193571,193573,193581,193583,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,2.5,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
print('Ratings matrix has {} rows and {} coulmns'.format(ratings.shape[0], ratings.shape[1]))

Ratings matrix has 610 rows and 8980 coulmns


In [53]:
# Now it would be useful to normalize the ratings around the average of each user (row)
# First we need to make averages array

average_ratings = ratings.mean(axis = 1)
average_ratings

userId
1      4.351351
2      3.888889
3      2.661290
4      3.554348
5      3.705882
         ...   
606    3.671683
607    3.780822
608    3.123672
609    3.300000
610    3.692345
Length: 610, dtype: float64

In [54]:
# Normalizing the ratings matrix

normalized_ratings = ratings.apply(lambda x: x - average_ratings[x.name], axis = 1)
normalized_ratings

movieId,1,2,3,4,5,6,7,8,9,10,...,190221,191005,193565,193567,193571,193573,193581,193583,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,-0.351351,NaN,-0.351351,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.294118,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,-1.171683,NaN,NaN,NaN,NaN,NaN,-1.171683,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,0.219178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,-0.623672,-1.123672,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.876328,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
# Make a numpy ratings matrix for 'easier' use

R = np.zeros((610,193609))

tmp = normalized_ratings.unstack().reset_index(name='rating')

tmp = tmp[tmp['rating'].notna()]

for index,row in tmp.iterrows():
    R[row['userId'].astype(np.int)-1][row['movieId'].astype(np.int)-1] = row['rating'] 
R 

array([[-0.35135135,  0.        , -0.35135135, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.62367223, -1.12367223,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.3       ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.30765504,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [63]:
def intersection(u,v):
    _u, _v = [], []
    for i in range(len(u)):
        if u[i] is not 0 and v[i] is not 0:
            _u.append(u[i])
            _v.append(v[i])
    return _u, _v

In [64]:
def find_similar_users(u):
    sims = []
    for v in range(610):
        if v is not u:
            a, b= intersection(R[u], R[v])
            sims.append((v,pearsonr(a, b)[0]))
    sims = sorted(sims, key=lambda x: x[1])
    sims.reverse()
    return sims[:5]

In [65]:
def find_similar_users_who_have_rated_specific_item(u, j):
    sims = []
    for v in range(610):
        if v is not u and R[v][j] != 0:
            a, b = intersection(R[u], R[v])
            sims.append((v,pearsonr(a, b)[0]))
    sims = sorted(sims, key=lambda x: x[1])
    sims.reverse()
    return sims[:5]

In [66]:
find_similar_users_who_have_rated_specific_item(0, 1)

[(589, 0.10954574503239975),
 (476, 0.10520393161801654),
 (433, 0.08717252161168262),
 (413, 0.07743849837092051),
 (134, 0.069725926835354)]

In [67]:
averages = []
for row in average_ratings:
    averages.append(row)

In [68]:
def predict_rating(u, j):
    r_uj = averages[u]
    num, den = 0, 0
    for v, sim in find_similar_users_who_have_rated_specific_item(u, j):
        num += R[v][j] * sim
        den += abs(sim)
    try:
        return r_uj + (num / den)
    except ZeroDivisionError:
        return -10

In [69]:
predict_rating(0, 0)

4.610356791721543

In [77]:
for index, row in test_data.iterrows():
    if row['userId'] <= 75:
        print("User Id : ", row['userId'], ", Movie Id : ", row['movieId'], ", Movie Title : ",
             row['title'])
        print("Predicted rating : ",predict_rating(row['userId'] - 1, row['movieId'] - 1), " --- Observed rating : ", row['rating'])

User Id :  21 , Movie Id :  70286 , Movie Title :  District 9 (2009)
Predicted rating :  2.8008523103251832  --- Observed rating :  3.5
User Id :  16 , Movie Id :  260 , Movie Title :  Star Wars: Episode IV - A New Hope (1977)
Predicted rating :  3.86050948636523  --- Observed rating :  3.0
User Id :  68 , Movie Id :  2797 , Movie Title :  Big (1988)
Predicted rating :  3.2547222610740763  --- Observed rating :  3.0
User Id :  62 , Movie Id :  86911 , Movie Title :  Hangover Part II, The (2011)
Predicted rating :  4.328445351465426  --- Observed rating :  4.0
User Id :  41 , Movie Id :  2712 , Movie Title :  Eyes Wide Shut (1999)
Predicted rating :  3.528152738107245  --- Observed rating :  5.0
User Id :  64 , Movie Id :  1967 , Movie Title :  Labyrinth (1986)
Predicted rating :  4.1551402410362765  --- Observed rating :  4.5
User Id :  18 , Movie Id :  260 , Movie Title :  Star Wars: Episode IV - A New Hope (1977)
Predicted rating :  4.566628864129117  --- Observed rating :  4.0
User 

KeyboardInterrupt: 